# Les 9: Verwachtingswaarde en Variantie

**Mathematical Foundations - IT & Artificial Intelligence**

---

## 9.0 Recap en Motivatie

In les 8 hebben we kansverdelingen geleerd: manieren om onzekerheid te beschrijven. Maar een volledige verdeling kan complex zijn. Vaak willen we een verdeling samenvatten in slechts enkele getallen.

De twee belangrijkste samenvattende maten zijn:
- **Verwachtingswaarde (gemiddelde)**: het "centrum" van de verdeling
- **Variantie (spreiding)**: hoe breed de verdeling is

In machine learning zijn deze concepten overal:
- Batch normalization gebruikt gemiddelde en variantie
- De MSE loss is gebaseerd op verwachtingswaarde
- Regularisatie beïnvloedt de variantie van het model
- Weight initialisatie is ontworpen om variantie te controleren

## 9.1 Leerdoelen

Na deze les kun je de verwachtingswaarde berekenen voor discrete en continue verdelingen. Je begrijpt variantie en standaarddeviatie als maten voor spreiding. Je kunt covariantie en correlatie berekenen. Je begrijpt hoe deze concepten worden gebruikt in neurale netwerken.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

np.set_printoptions(precision=4, suppress=True)
np.random.seed(42)

print("Libraries geladen!")

## 9.2 Verwachtingswaarde

### Definitie

De verwachtingswaarde (of expected value) E[X] is het "gewogen gemiddelde" van alle mogelijke uitkomsten, gewogen naar hun kans.

**Discreet:**
E[X] = Σ x · P(X = x)

**Continu:**
E[X] = ∫ x · f(x) dx

### Intuïtie

Als je het experiment oneindig vaak zou herhalen en het gemiddelde zou nemen van alle uitkomsten, krijg je de verwachtingswaarde.

In [ ]:
# Voorbeeld: verwachtingswaarde van een dobbelsteen
outcomes = np.array([1, 2, 3, 4, 5, 6])
probabilities = np.array([1/6] * 6)

# E[X] = Σ x · P(X = x)
expected_value = np.sum(outcomes * probabilities)

print(f"Dobbelsteen: E[X] = {expected_value}")
print(f"Berekening: (1+2+3+4+5+6)/6 = {sum(outcomes)/6}")

# Verificatie met simulatie
n_throws = 100000
throws = np.random.randint(1, 7, n_throws)
sample_mean = np.mean(throws)

print(f"\nSimulatie ({n_throws} worpen): gemiddelde = {sample_mean:.4f}")

In [ ]:
# Niet-eerlijke dobbelsteen: hogere kans op 6
unfair_probs = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.5])  # 50% kans op 6!

E_unfair = np.sum(outcomes * unfair_probs)
print(f"Niet-eerlijke dobbelsteen: E[X] = {E_unfair}")
print(f"Hoger dan eerlijk (3.5) omdat 6 waarschijnlijker is.")

# Visualisatie
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].bar(outcomes, probabilities, color='steelblue', edgecolor='black')
axes[0].axvline(x=3.5, color='red', linestyle='--', linewidth=2, label=f'E[X] = 3.5')
axes[0].set_xlabel('Uitkomst')
axes[0].set_ylabel('Kans')
axes[0].set_title('Eerlijke dobbelsteen')
axes[0].legend()

axes[1].bar(outcomes, unfair_probs, color='steelblue', edgecolor='black')
axes[1].axvline(x=E_unfair, color='red', linestyle='--', linewidth=2, label=f'E[X] = {E_unfair}')
axes[1].set_xlabel('Uitkomst')
axes[1].set_ylabel('Kans')
axes[1].set_title('Niet-eerlijke dobbelsteen')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Verwachtingswaarde van continue verdeling: Normaal
mu, sigma = 5, 2

# E[X] = μ voor normale verdeling
print(f"Normale verdeling N({mu}, {sigma}²): E[X] = {mu}")

# Verificatie met samples
samples = np.random.normal(mu, sigma, 100000)
print(f"Sample gemiddelde: {np.mean(samples):.4f}")

# Visualisatie
x = np.linspace(-2, 12, 1000)
pdf = stats.norm.pdf(x, mu, sigma)

plt.figure(figsize=(10, 5))
plt.plot(x, pdf, 'b-', linewidth=2)
plt.fill_between(x, pdf, alpha=0.3)
plt.axvline(x=mu, color='red', linestyle='--', linewidth=2, label=f'E[X] = μ = {mu}')
plt.xlabel('x', fontsize=12)
plt.ylabel('f(x)', fontsize=12)
plt.title('Verwachtingswaarde van N(5, 4)', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

### Eigenschappen van Verwachtingswaarde

1. **Lineariteit**: E[aX + b] = a·E[X] + b
2. **Additiviteit**: E[X + Y] = E[X] + E[Y] (altijd waar!)
3. **Product**: E[XY] = E[X]·E[Y] alleen als X en Y onafhankelijk zijn

In [ ]:
# Demonstratie van lineariteit: E[2X + 3]
X = np.random.normal(5, 2, 100000)

E_X = np.mean(X)
E_2X_plus_3 = np.mean(2*X + 3)

print("Lineariteit: E[aX + b] = a·E[X] + b")
print(f"E[X] = {E_X:.4f}")
print(f"E[2X + 3] = {E_2X_plus_3:.4f}")
print(f"2·E[X] + 3 = {2*E_X + 3:.4f}")

## 9.3 Variantie

### Definitie

De variantie Var(X) meet hoe ver de waarden gemiddeld afwijken van de verwachtingswaarde:

Var(X) = E[(X - E[X])²] = E[X²] - (E[X])²

### Standaarddeviatie

De standaarddeviatie σ = √Var(X) is in dezelfde eenheid als X en is intuïtiever.

### Intuïtie

- Kleine variantie: waarden clusteren dicht bij het gemiddelde
- Grote variantie: waarden zijn meer verspreid

In [ ]:
# Variantie van een dobbelsteen
outcomes = np.array([1, 2, 3, 4, 5, 6])
probabilities = np.array([1/6] * 6)

E_X = np.sum(outcomes * probabilities)
E_X2 = np.sum(outcomes**2 * probabilities)

# Var(X) = E[X²] - (E[X])²
Var_X = E_X2 - E_X**2
Std_X = np.sqrt(Var_X)

print(f"Dobbelsteen:")
print(f"  E[X] = {E_X}")
print(f"  E[X²] = {E_X2:.4f}")
print(f"  Var(X) = {Var_X:.4f}")
print(f"  σ = {Std_X:.4f}")

# Verificatie
throws = np.random.randint(1, 7, 100000)
print(f"\nSample variantie: {np.var(throws):.4f}")
print(f"Sample std: {np.std(throws):.4f}")

In [ ]:
# Vergelijk verdelingen met verschillende varianties
mu = 0
sigmas = [0.5, 1, 2, 4]

x = np.linspace(-10, 10, 1000)

plt.figure(figsize=(12, 5))

for sigma in sigmas:
    pdf = stats.norm.pdf(x, mu, sigma)
    plt.plot(x, pdf, linewidth=2, label=f'σ = {sigma} (Var = {sigma**2})')

plt.xlabel('x', fontsize=12)
plt.ylabel('f(x)', fontsize=12)
plt.title('Normale verdelingen met verschillende varianties', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print("Grotere variantie = bredere, plattere verdeling")

### Eigenschappen van Variantie

1. **Constante**: Var(aX + b) = a²·Var(X) (constante b verdwijnt!)
2. **Som (onafhankelijk)**: Var(X + Y) = Var(X) + Var(Y) als X, Y onafhankelijk
3. **Niet-negativiteit**: Var(X) ≥ 0, met gelijkheid alleen als X constant is

In [ ]:
# Demonstratie: Var(aX + b) = a²·Var(X)
X = np.random.normal(5, 2, 100000)  # Var(X) = 4

a, b = 3, 10
Y = a*X + b

print("Eigenschap: Var(aX + b) = a²·Var(X)")
print(f"Var(X) = {np.var(X):.4f}")
print(f"Var({a}X + {b}) = {np.var(Y):.4f}")
print(f"{a}²·Var(X) = {a**2 * np.var(X):.4f}")
print("\nMerk op: de constante b heeft geen effect op de variantie!")

## 9.4 Covariantie en Correlatie

### Covariantie

De covariantie meet hoe twee variabelen samen variëren:

Cov(X, Y) = E[(X - E[X])(Y - E[Y])] = E[XY] - E[X]·E[Y]

- Cov > 0: X en Y bewegen samen omhoog/omlaag
- Cov < 0: als X omhoog gaat, gaat Y omlaag
- Cov = 0: geen lineair verband

### Correlatie

De correlatie is de genormaliseerde covariantie:

ρ(X, Y) = Cov(X, Y) / (σ_X · σ_Y)

De correlatie ligt altijd tussen -1 en 1.

In [ ]:
# Demonstratie van correlatie
n = 500

# Positief gecorreleerd
X1 = np.random.randn(n)
Y1 = X1 + np.random.randn(n) * 0.3  # Y = X + noise

# Negatief gecorreleerd
X2 = np.random.randn(n)
Y2 = -X2 + np.random.randn(n) * 0.3

# Ongecorreleerd
X3 = np.random.randn(n)
Y3 = np.random.randn(n)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, (X, Y), title in zip(axes, [(X1,Y1), (X2,Y2), (X3,Y3)], 
                               ['Positief', 'Negatief', 'Geen']):
    corr = np.corrcoef(X, Y)[0, 1]
    ax.scatter(X, Y, alpha=0.5)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title(f'{title} gecorreleerd\nρ = {corr:.3f}')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Covariantie matrix
# Voor meerdere variabelen gebruiken we een covariantie matrix

# Genereer gecorreleerde data
mean = [0, 0]
cov_matrix = [[1, 0.8],   # Var(X)=1, Cov(X,Y)=0.8
              [0.8, 1]]   # Cov(Y,X)=0.8, Var(Y)=1

data = np.random.multivariate_normal(mean, cov_matrix, 1000)
X, Y = data[:, 0], data[:, 1]

# Bereken sample covariantie matrix
sample_cov = np.cov(data.T)
sample_corr = np.corrcoef(data.T)

print("Populatie covariantie matrix:")
print(np.array(cov_matrix))
print("\nSample covariantie matrix:")
print(sample_cov)
print("\nSample correlatie matrix:")
print(sample_corr)

## 9.5 Toepassing: Batch Normalization

Batch Normalization is een techniek die gemiddelde en variantie gebruikt om training te stabiliseren.

Voor een batch activaties x:
1. Bereken batch gemiddelde: μ_B = (1/m) Σ x_i
2. Bereken batch variantie: σ²_B = (1/m) Σ (x_i - μ_B)²
3. Normaliseer: x̂_i = (x_i - μ_B) / √(σ²_B + ε)
4. Scale en shift: y_i = γ·x̂_i + β

Dit zorgt ervoor dat activaties gemiddelde ≈ 0 en variantie ≈ 1 hebben.

In [ ]:
class BatchNorm:
    """Simpele Batch Normalization implementatie."""
    
    def __init__(self, n_features, epsilon=1e-5):
        self.epsilon = epsilon
        self.gamma = np.ones(n_features)  # Scale parameter
        self.beta = np.zeros(n_features)   # Shift parameter
    
    def forward(self, x):
        """Normaliseer een batch."""
        # x shape: (batch_size, n_features)
        self.mu = np.mean(x, axis=0)
        self.var = np.var(x, axis=0)
        
        # Normaliseer
        self.x_norm = (x - self.mu) / np.sqrt(self.var + self.epsilon)
        
        # Scale en shift
        out = self.gamma * self.x_norm + self.beta
        return out

# Demonstratie
np.random.seed(42)

# Simuleer activaties met groot gemiddelde en variantie
batch_size = 64
n_features = 4
activations = np.random.randn(batch_size, n_features) * 10 + 50  # Grote mean, grote var

print("Vóór BatchNorm:")
print(f"  Gemiddelde per feature: {np.mean(activations, axis=0)}")
print(f"  Variantie per feature: {np.var(activations, axis=0)}")

bn = BatchNorm(n_features)
normalized = bn.forward(activations)

print("\nNa BatchNorm:")
print(f"  Gemiddelde per feature: {np.mean(normalized, axis=0)}")
print(f"  Variantie per feature: {np.var(normalized, axis=0)}")

In [ ]:
# Visualiseer het effect
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Voor BatchNorm
axes[0].hist(activations.flatten(), bins=30, alpha=0.7, density=True)
axes[0].axvline(x=np.mean(activations), color='red', linestyle='--', label=f'μ = {np.mean(activations):.1f}')
axes[0].set_xlabel('Activatie')
axes[0].set_ylabel('Dichtheid')
axes[0].set_title('Vóór BatchNorm')
axes[0].legend()

# Na BatchNorm
axes[1].hist(normalized.flatten(), bins=30, alpha=0.7, density=True)
axes[1].axvline(x=np.mean(normalized), color='red', linestyle='--', label=f'μ ≈ {np.mean(normalized):.2f}')
axes[1].set_xlabel('Activatie')
axes[1].set_ylabel('Dichtheid')
axes[1].set_title('Na BatchNorm')
axes[1].legend()

plt.tight_layout()
plt.show()

## 9.6 Toepassing: Xavier/He Initialisatie

De variantie van weight initialisatie is cruciaal voor goede training.

### Xavier Initialisatie (voor tanh/sigmoid)
W ~ N(0, 2/(n_in + n_out))

### He Initialisatie (voor ReLU)
W ~ N(0, 2/n_in)

Het doel is om de variantie van activaties constant te houden door de lagen.

In [ ]:
# Demonstratie: effect van weight initialisatie op variantie
def relu(x):
    return np.maximum(0, x)

def forward_pass(x, weights):
    """Forward pass door meerdere lagen met ReLU."""
    activations = [x]
    for W in weights:
        x = relu(x @ W)
        activations.append(x)
    return activations

# Netwerk: 256 -> 256 -> 256 -> 256 -> 256 (5 lagen)
n_layers = 5
n_neurons = 256
batch_size = 100

# Input
x = np.random.randn(batch_size, n_neurons)

# Slechte initialisatie: te kleine variantie
weights_small = [np.random.randn(n_neurons, n_neurons) * 0.01 for _ in range(n_layers)]
acts_small = forward_pass(x, weights_small)

# Slechte initialisatie: te grote variantie
weights_large = [np.random.randn(n_neurons, n_neurons) * 1.0 for _ in range(n_layers)]
acts_large = forward_pass(x, weights_large)

# He initialisatie: juiste variantie
weights_he = [np.random.randn(n_neurons, n_neurons) * np.sqrt(2/n_neurons) for _ in range(n_layers)]
acts_he = forward_pass(x, weights_he)

# Plot varianties
fig, ax = plt.subplots(figsize=(10, 6))

layers = range(n_layers + 1)
ax.plot(layers, [np.var(a) for a in acts_small], 'r-o', label='Te kleine var (0.01)')
ax.plot(layers, [np.var(a) for a in acts_large], 'b-o', label='Te grote var (1.0)')
ax.plot(layers, [np.var(a) for a in acts_he], 'g-o', label='He initialisatie')

ax.set_xlabel('Laag', fontsize=12)
ax.set_ylabel('Variantie van activaties', fontsize=12)
ax.set_title('Effect van weight initialisatie op activatie variantie', fontsize=14)
ax.set_yscale('log')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.show()

print("Te kleine variantie → activaties krimpen naar 0 (vanishing)")
print("Te grote variantie → activaties exploderen (exploding)")
print("He initialisatie → stabiele variantie door alle lagen")

## 9.7 Samenvatting

### Kernconcepten

**Verwachtingswaarde E[X]** is het gewogen gemiddelde van alle mogelijke uitkomsten. Het is lineair: E[aX + b] = a·E[X] + b.

**Variantie Var(X)** meet de spreiding rond het gemiddelde. Het is kwadratisch: Var(aX + b) = a²·Var(X).

**Covariantie en correlatie** meten hoe twee variabelen samen variëren. Correlatie is genormaliseerd naar [-1, 1].

### Toepassingen in ML

- **Batch Normalization**: normaliseert activaties naar gemiddelde 0 en variantie 1
- **Weight initialisatie**: kiest variantie om stabiele training te garanderen
- **Loss functions**: MSE is gebaseerd op verwachte kwadratische fout

### Volgende les

In les 10 leren we over Maximum Likelihood Estimation: de theoretische basis voor het trainen van modellen.

---

**Mathematical Foundations** | Les 9 van 12 | IT & Artificial Intelligence

---